# Preparation: library loading

In [1]:
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from scipy.misc import imread

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.pipeline import Pipeline
from sklearn import base

from joblib import dump, load
import pickle
import dill

In [3]:
import nltk
import re

nltk.download('stopwords');
nltk.download('punkt');

import spacy
nlp = spacy.load('en');

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['the','i','you','a','c','slu','amazon','google','apple','microsoft','company'])

In [5]:
def tokenization(text):
    '''
    INPUT: a single user comment, eg. 'Work hard, have fun, make history. Be proud of our self, the job we are doing.'
    OUTPUT: tokenized comment (i.e, single words), eg. ['work','hard','have','fun', 'make','history',...]
    
    NOTE:
    nltk.sent_tokenize: this gives a list of sentences
    nltk.word_tokenize: this gives a list of sentences
    '''
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.lower() not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('^[a-z|A-Z]+$', token):
            filtered_tokens.append(token)
    return filtered_tokens

### Dump & Load

In [121]:
# Dump
keyword = open("Keyword.pkl","wb")
pickle.dump(KW_dict,keyword)
keyword.close()

topic = open("Topic.pkl","wb")
pickle.dump(Topic_dict,topic)
topic.close()

In [128]:
keyword_s = open("Keyword_s.pkl","wb")
pickle.dump(KW_dict_s,keyword_s)
keyword_s.close()

In [47]:
# Load
KW_dict = pickle.load(open("Keyword.pkl", "rb"))

Topic_dict = pickle.load(open("Topic.pkl", "rb"))

In [48]:
KW_dict['google']

'amazing balance benefits best culture environment food free free food good great life nice opportunities pay people perks place salary smart smart people work working'

In [49]:
Topic_dict['amazon']

['Topic 0: work culture',
 'Topic 1: good pay',
 'Topic 2: people get',
 'Topic 3: benefits great']

# Part 0: Data Import

In [8]:
df = pd.read_csv(r'./all_company.csv')
print('The dimension of the table is ' + str(df.shape[0]) + ' rows and ' + str(df.shape[1]) + ' columns.')  

The dimension of the table is 122249 rows and 18 columns.


In [29]:
company_list=df.groupby('Company').count().index.tolist()

In [30]:
stopwords.extend(company_list)

In [31]:
len(company_list)

107

# Part 1: ViolinPlots, Keyword, WordClouds

In [122]:
def get_Violin(target_company, df):
    '''
    Input: target_company => 'google'
    Output: text_pros, violinplots(save into file)
    '''
    # filter dataframe
    df_t=df.loc[df['Company'] == target_company]
    df_t=df_t.rename(columns={'rating_overall':'Overall', "rating_balance":"Work-Life Balance", 
                         "rating_culture": "Culture","rating_career":"Career", 
                          "rating_comp":"Compensation","rating_mgmt":"Management"})
    
    
    # Violin plot
    #plt.clf()
    sns.set(rc={'figure.figsize':(18,6)});
    Vplot = sns.violinplot(data=df_t[['Overall','Work-Life Balance','Culture','Career',
                                  'Compensation','Management']],palette='Set3');
    Vplot.set_title(target_company.capitalize()+' Ratings Plot', fontsize=30);
    Vplot.set(xlabel='Ratings', ylabel='Stars');
    Vplot.set_xlabel('Ratings', fontsize=20);
    Vplot.set_ylabel('Stars', fontsize=20);
    Vplot.figure.savefig(r'./ViolinPlots/{}_Violin.jpg'.format(target_company));
    plt.clf()
    return



In [123]:
for i in range(len(company_list)):
    get_Violin(company_list[i],df)

<Figure size 1296x432 with 0 Axes>

In [115]:
# ViolinPlots & Keyword_dict
def get_keyword(target_company, df):
    '''
    Input: target_company => 'google'
    Output: text_pros, violinplots(save into file)
    '''
    # filter dataframe
    df_t=df.loc[df['Company'] == target_company]
    
    '''
    # Violin plot
    sns.set(rc={'figure.figsize':(18,6)});
    Vplot = sns.violinplot(data=df_t[['rating_overall','rating_balance','rating_culture','rating_career',
                                  'rating_comp','rating_mgmt']],palette='Set3');
    Vplot.set_title(target_company.capitalize()+' Ratings Plot', fontsize=30);
    Vplot.set(xlabel='Ratings', ylabel='Stars');
    Vplot.set_xlabel('Ratings', fontsize=20,fontname="Arial");
    Vplot.set_ylabel('Stars', fontsize=20, fontname="Arial");
    Vplot.figure.savefig(r'./ViolinPlots/{}_Violin.jpg'.format(target_company));
    plt.clf()
    '''
    
    # feature words of pros
    def get_pros(df_company):
        pros = []
        for i in range(len(df_company)):  
            pros.append(df_company.iloc[i][9])
        return pros
    
    pros = get_pros(df_t)
    
    tfidf_model = TfidfVectorizer(max_df=0.75, max_features=100,
                                 min_df=0.05, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenization, ngram_range=(1,2))
    tfidf_matrix = tfidf_model.fit_transform(pros)
    feature_pros = tfidf_model.get_feature_names()
    text_pros = ' '.join(feature_pros)
    
    return text_pros

'\nKW_dict = {}\nfor i in range(len(company_list)):\n    KW_dict[company_list[i]]=get_keyword(company_list[i],df)\n'

In [ ]:
KW_dict = {}
for i in range(len(company_list)):
    KW_dict[company_list[i]]=get_keyword(company_list[i],df)

In [127]:
KW_dict_s = {}
company_list_s = ['google','amazon','point72','microsoft','apple', 'accenture', 'airbnb','altria',
               'boeing','broadcom','capitalone','citi','comcast','deloitte','dupont','facebook','honeywell','intel',
                'lyft','netflix','nike','pfizer','target','uber','visa','verizon','linkedin','bayer','chevron']
for i in range(len(company_list_s)):
    KW_dict_s[company_list_s[i]]=get_keyword(company_list_s[i],df)

In [ ]:
# WordClouds
for k, v in KW_dict.items():
    config = Path(r'./Masks/mask_{}.png'.format(k))
    if config.is_file():
        mask = imread(r'./Masks/mask_{}.png'.format(k))
        wc=WordCloud(mask=mask,background_color="white",scale=2,repeat=True,colormap = 'viridis').generate(v)
        wc.to_file(r'./WordClouds/{}_wordcloud.jpg'.format(k))
    else:
        mask = imread(r'./Masks/mask.png'.format(k))
        wc=WordCloud(mask=mask,background_color="white",scale=2,repeat=True,colormap = 'viridis').generate(v)
        wc.to_file(r'./WordClouds/{}_wordcloud.jpg'.format(k))

# Part 2: Topic Modelling

In [33]:
def get_topic(target_company, df):
    '''
    Input: target_company
    Output: first 4 topic of each company
    '''
    class ReviewTransformer(base.BaseEstimator, base.TransformerMixin):
        def __init__(self, target_company):
            self.name = target_company  # We will need these in transform()

        def fit(self, X, y=None):
            # This transformer doesn't need to learn anything about the data,
            # so it can just return self without any further processing
            return self

        def transform(self, X):
            # Return an array with the same number of rows as X and one
            # column for each in self.col_names

            X_t=X.loc[X['Company'] == self.name]

            pros = []
            for i in range(len(X_t)):  
                pros.append(X_t.iloc[i][9])
            return pros
        
    CntVec = CountVectorizer(max_df=0.75, min_df=0.05, max_features=100, 
                             stop_words=stopwords, tokenizer=tokenization, ngram_range=(1,2))
    
    LDA = LatentDirichletAllocation(n_components=4, max_iter=5, learning_method='online', 
                                    learning_offset=50.,random_state=0)
    
    lda_est = Pipeline([('GetProsCmt', ReviewTransformer(target_company)),
                        ('CountVectorizer', CntVec),
                        ('LatentDirichletAllocation', LDA)])
    
    model = lda_est.fit(df)
    
    def display_topics(model, feature_names, no_top_words):
        res = []
        for topic_idx, topic in enumerate(model.components_):
            topic_words = " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
            elm = "Topic {}: {}".format(topic_idx, topic_words)
            res.append(elm)
            #print(elm)
        return res
    
    topics = display_topics(lda_est.named_steps['LatentDirichletAllocation'], 
                            lda_est.named_steps['CountVectorizer'].get_feature_names(), 2)
    return topics,model

In [37]:
target_company = 'google'
Topic,model=get_topic(target_company,df)
dill.dump(model, open(r'./TopicModel/{}_TopicModel.joblib'.format(target_company), 'wb'))

lda_est = model
step1 = lda_est.named_steps['GetProsCmt'].fit_transform(df)
step2 = lda_est.named_steps['CountVectorizer'].fit_transform(step1)
step3 = lda_est.named_steps['LatentDirichletAllocation'].fit(step2)
topic_plot = pyLDAvis.sklearn.prepare(step3, step2, lda_est.named_steps['CountVectorizer'])
pyLDAvis.save_html(topic_plot, r'./LDA_plot/{}.html'.format(target_company))

In [39]:
Topic_dict = {}
for i in range(len(company_list)):
    target_company = company_list[i]
    Topic_dict[company_list[i]],model=get_topic(company_list[i],df)
    dill.dump(model, open(r'./TopicModel/{}_TopicModel.joblib'.format(target_company), 'wb'))
    
    
    lda_est = model
    step1 = lda_est.named_steps['GetProsCmt'].fit_transform(df)
    step2 = lda_est.named_steps['CountVectorizer'].fit_transform(step1)
    step3 = lda_est.named_steps['LatentDirichletAllocation'].fit(step2)
    topic_plot = pyLDAvis.sklearn.prepare(step3, step2, lda_est.named_steps['CountVectorizer'])
    pyLDAvis.save_html(topic_plot, r'./LDA_plot/{}.html'.format(target_company))
    

In [ ]:
Topic_dict['amazon'] # type list

### Save Model (pipeline)

In [19]:
dill.dump(model, open(r'./TopicModel/{}_TopicModel.joblib'.format(target_company), 'wb'))

In [20]:
lda_est = dill.load(open(r'./TopicModel/{}_TopicModel.joblib'.format(target_company), 'rb'))

### display plot

In [21]:
import pyLDAvis.sklearn
from __future__ import print_function
pyLDAvis.enable_notebook()

In [22]:
step1 = lda_est.named_steps['GetProsCmt'].fit_transform(df)

In [23]:
step2 = lda_est.named_steps['CountVectorizer'].fit_transform(step1)

In [24]:
step3 = lda_est.named_steps['LatentDirichletAllocation'].fit(step2)

In [25]:
topic_plot = pyLDAvis.sklearn.prepare(step3, step2, lda_est.named_steps['CountVectorizer'])
topic_plot

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.124253  0.176325       1        1  31.413186
3     -0.232925 -0.188291       2        1  24.569159
1      0.095483  0.150890       3        1  22.919171
2      0.261696 -0.138925       4        1  21.098484, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
9     Default  549.000000           good  549.000000  23.0000  23.0000
21    Default  570.000000           work  570.000000  22.0000  22.0000
15    Default  469.000000         people  469.000000  21.0000  21.0000
6     Default  264.000000           food  264.000000  20.0000  20.0000
19    Default  188.000000          smart  188.000000  19.0000  19.0000
7     Default  174.000000           free  174.000000  18.0000  18.0000
16    Default  280.000000          perks  280.000000  17.0000  17.0000
5     Default  222.000000    environment  222.000000  16.0000  16.0000
2     Default  300.000000       benefits  300.000000  15.0000  15.0000
20    Default  109.000000   smart people  109.000000  14.0000  14.0000
10    Default  700.000000          great  700.000000  13.0000  13.0000
8     Default  108.000000      free food  108.000000  12.0000  12.0000
18    Default  110.000000         salary  110.000000  11.0000  11.0000
17    Default  134.000000          place  134.000000  10.0000  10.0000
1     Default  120.000000        balance  120.000000   9.0000   9.0000
3     Default  112.000000           best  112.000000   8.0000   8.0000
14    Default  107.000000            pay  107.000000   7.0000   7.0000
12    Default  129.000000           nice  129.000000   6.0000   6.0000
22    Default  144.000000        working  144.000000   5.0000   5.0000
0     Default  111.000000        amazing  111.000000   4.0000   4.0000
11    Default   92.000000           life   92.000000   3.0000   3.0000
13    Default   99.000000  opportunities   99.000000   2.0000   2.0000
4     Default  235.000000        culture  235.000000   1.0000   1.0000
11     Topic1   91.565997           life   92.646995   1.1462  -2.9078
3      Topic1  111.351657           best  112.772324   1.1453  -2.7122
1      Topic1  118.236566        balance  120.051720   1.1427  -2.6522
17     Topic1  131.450037          place  134.130559   1.1378  -2.5462
21     Topic1  548.074695           work  570.548402   1.1178  -1.1184
13     Topic1   94.455282  opportunities   99.928763   1.1016  -2.8767
10     Topic1  324.634847          great  700.537422   0.3888  -1.6422
...       ...         ...            ...         ...      ...      ...
11     Topic3    0.390567           life   92.646995  -3.9958  -8.0498
1      Topic3    0.471725        balance  120.051720  -4.0661  -7.8610
18     Topic3    0.381491         salary  110.378647  -4.1944  -8.0733
17     Topic3    0.349988          place  134.130559  -4.4755  -8.1595
9      Topic3    1.307111           good  549.971757  -4.5689  -6.8418
2      Topic3    0.635729       benefits  300.593664  -4.6855  -7.5626
5      Topic3    0.356299    environment  222.767301  -4.9649  -8.1416
20     Topic4  107.956836   smart people  109.336428   1.5433  -2.3451
19     Topic4  185.558227          smart  188.228283   1.5417  -1.8035
0      Topic4   84.678930        amazing  111.768929   1.2784  -2.5880
5      Topic4  166.217377    environment  222.767301   1.2631  -1.9135
15     Topic4  343.388624         people  469.945681   1.2422  -1.1880
22     Topic4   87.688893        working  144.519016   1.0564  -2.5530
12     Topic4   20.854190           nice  129.278354  -0.2684  -3.9893
2      Topic4   43.018433       benefits  300.593664  -0.3882  -3.2652
16     Topic4   22.115253          perks  280.320463  -0.9837  -3.9306
9      Topic4   28.261036           good  549.971757  -1.4124  -3.6853
13     Topic4    3.795414  opportunities   99.928763  -1.7147  -5.6930
4      Topic

In [26]:
pyLDAvis.save_html(topic_plot, r'./LDA_plot/{}.html'.format(target_company))

# Part 6: New Recommendation

In [119]:
def get_recommendation(user_prf, KW_dict):
    
    def stopword_RMV(sent):
        res = []
        for word in sent.split():
            if word.lower() not in stopwords:
                res.append(word)
        return ' '.join(res)
    
    doc0 = nlp(stopword_RMV(user_prf))
    score_dict = {}
    for k,v in KW_dict.items():
        temp_doc = nlp(v)
        score_dict[k]= doc0.similarity(temp_doc)
        
    sorted_score = sorted(score_dict.items(), key=lambda kv: kv[1],reverse=True)
    
    rcm_company = []
    for i in range(5):
        rcm_company.append('#'+str(i+1)+': '+str(sorted_score[i][0].capitalize()))

    return rcm_company

In [120]:
rcm_company=get_recommendation(input('Your Preference:'),KW_dict_s)
rcm_company

Your Preference:free food


['#1: Mcdonald', '#2: Google', '#3: Netflix', '#4: Facebook', '#5: Point72']